In [1]:
import h5py as h5
import numpy as np
from scipy.integrate import quad

import astropy.units as u
import astropy.constants as c
import cogsworth

import matplotlib.pyplot as plt
from importlib import reload
import pandas as pd
import functools

import os

import sys
# sys.path.append("../src/")
from compas_processing import mask_COSMIC_data, get_COSMIC_vars
from galaxy import simulate_mw, simulate_simple_mw
from variations import variations

%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='serif')
fs = 20

# Calculate mass fraction sampled

### Using https://github.com/TeamCOMPAS/COMPAS/blob/dev/compas_python_utils/cosmic_integration/totalMassEvolvedPerZ.py#L55

In [2]:
@functools.lru_cache()
def __get_imf_normalisation_values(m1=0.01, m2=0.08, m3=0.5, m4=200.0, a12=0.3, a23=1.3, a34=2.3):
    b1 = 1 / (
            (m2 ** (1 - a12) - m1 ** (1 - a12)) / (1 - a12)
            + m2 ** (-(a12 - a23)) * (m3 ** (1 - a23) - m2 ** (1 - a23)) / (1 - a23)
            + m2 ** (-(a12 - a23)) * m3 ** (-(a23 - a34)) * (m4 ** (1 - a34) - m3 ** (1 - a34)) / (1 - a34)
    )
    b2 = b1 * m2 ** (-(a12 - a23))
    b3 = b2 * m3 ** (-(a23 - a34))
    return b1, b2, b3

@np.vectorize
def IMF(m, m1=0.01, m2=0.08, m3=0.5, m4=200.0, a12=0.3, a23=1.3, a34=2.3):
    """Calculate the fraction of stellar mass between m and m + dm for a three part broken power law.

    Default values follow Kroupa (2001)
    https://arxiv.org/abs/astro-ph/0009005
    Equation 1-2

            zeta(m) ~ m^(-a_ij)
    Parameters
    ----------
    m : `float` or `np.ndarray`
        Mass at which to evaluate
    mi : float, optional
        masses at which to transition the slope
    aij : float, optional
        slope of the IMF between mi and mj
    Returns
    -------
    imf_vals
        IMF evaluated at the given masses
    """
    # calculate normalisation constants that ensure the IMF is continuous
    b1, b2, b3 = __get_imf_normalisation_values(m1, m2, m3, m4, a12, a23, a34)

    # evaluate IMF either at a point or for a list of points
    if m1 <= m < m2:
        return b1 * m ** (-a12)
    elif m2 <= m < m3:
        return b2 * m ** (-a23)
    elif m3 <= m < m4:
        return b3 * m ** (-a34)
    else:
        return 0.0

def get_cosmic_fraction(m1_low, m1_upp, m2_low, f_bin, mass_ratio_pdf_function=lambda q: 1,
                        m1=0.01, m2=0.08, m3=0.5, m4=200.0, a12=0.3, a23=1.3, a34=2.3):
    """Calculate the fraction of mass in a cosmic population relative to the total Universal population. This
    can be used to normalise the rates of objects from cosmic simulations.

    Parameters
    ----------
    m1_low : `float`
        Lower limit on the sampled primary mass
    m1_upp : `float`
        Upper limit on the sampled primary mass
    m2_low : `float`
        Lower limit on the sampled secondary mass
    f_bin : `float`
        Binary fraction
    mass_ratio_pdf_function : `function`, optional
        Function to calculate the mass ratio PDF, by default a uniform mass ratio distribution
    mi, aij : `float`
        Settings for the IMF choice, see `IMF` for details, by default follows Kroupa (2001)

    Returns
    -------
    fraction
        The fraction of mass in a cosmic population relative to the total Universal population
    """ 
    # first, for normalisation purposes, we can find the integral with no cosmic cuts
    def full_integral(mass, m1, m2, m3, m4, a12, a23, a34):
        primary_mass = IMF(mass, m1, m2, m3, m4, a12, a23, a34) * mass
        
        # find the expected companion mass given the mass ratio pdf function
        expected_secondary_mass = quad(lambda q: q * mass_ratio_pdf_function(q), 0, 1)[0] * primary_mass
        
        single_stars = (1 - f_bin) * primary_mass
        binary_stars = f_bin * (primary_mass + expected_secondary_mass)
        return single_stars + binary_stars
    full_mass = quad(full_integral, m1, m4, args=(m1, m2, m3, m4, a12, a23, a34))[0]
    
    # now we do a similar integral but for the cosmic regime
    def cosmic_integral(mass, m2_low, f_bin, m1, m2, m3, m4, a12, a23, a34):
        # define the primary mass in the same way
        primary_mass = IMF(mass, m1, m2, m3, m4, a12, a23, a34) * mass
        
        # find the fraction that are below the m2 mass cut
        f_below_m2low = quad(mass_ratio_pdf_function, 0, m2_low / mass)[0]
        
        # expectation value of the secondary mass given the m2 cut and mass ratio pdf function
        expected_secondary_mass = quad(lambda q: q * mass_ratio_pdf_function(q), m2_low / mass, 1)[0] * primary_mass
        
        # return total mass of binary stars that have m2 above the cut
        return f_bin * (1 - f_below_m2low) * (primary_mass + expected_secondary_mass)
    cosmic_mass = quad(cosmic_integral, m1_low, m1_upp, args=(m2_low, f_bin, m1, m2, m3, m4, a12, a23, a34))[0]
    return cosmic_mass / full_mass

# Get star forming mass per Z

In [3]:
def mass_per_Z(file, f_trunc, bins):
    initC = pd.read_hdf(file, key='full_initC')
    DCOs = initC.loc[initC.is_hit.values == 1]

    # Load weights
    weights = pd.read_hdf(file, key='weights')

    # Adjust weights to remove log uniform metallicity sampling (optional?)
    old_sum = np.sum(weights.mixture_weight.values)
    metallicities = initC.metallicity.values
    weights_adjustment = (0.03-0.0001)/((np.log10(0.03)-np.log10(0.0001))*metallicities)
    weights['mixture_weight'] /= weights_adjustment
    weights['mixture_weight'] *= old_sum / np.sum(weights.mixture_weight.values)
    
    # Filter weights for DCOs
    DCO_weights = weights.loc[initC.is_hit.values == 1]

    # Assign bins
    initC_bins = np.digitize(initC.metallicity.values, bins) - 1
    DCO_bins = np.digitize(DCOs.metallicity.values, bins) - 1

    # Sum weights for each bin to get DCOs per metallicity bin
    total_DCOs_per_Z = np.array([np.sum(DCO_weights.loc[DCO_bins == i].mixture_weight.values) for i in range(len(bins) - 1)])
    
    
    weighted_masses = (initC.mass_1.values + initC.mass_2.values) * weights.mixture_weight.values

    MSF_per_Z = np.array([np.sum(weighted_masses[initC_bins == i]) for i in range(len(bins) - 1)])

    MSF_per_Z /= f_trunc

    return MSF_per_Z, total_DCOs_per_Z


# Get metallicity weights from cogsworth galaxy

In [4]:
def find_metallicity_weights(bins):
    g = cogsworth.sfh.Wagg2022(size=10_000_000)

    sampled_Z = g.Z.value
    sampled_Z[sampled_Z > np.max(bins)] = np.max(bins)
    sampled_Z[sampled_Z < np.min(bins)] = np.min(bins)

    counts, _ = np.histogram(sampled_Z, bins=bins)
    
    MW_weights = counts / np.sum(counts)

    return MW_weights

In [5]:
# get bins
grid = np.linspace(-4, np.log10(3e-2), 50)
inner_bins = grid[:-1] + np.diff(grid) / 2
bins = 10**np.concatenate(([grid[0]], inner_bins, [grid[-1]]))

btypes = ['BHBH', 'BHNS', 'NSNS', 'BHWD', 'NSWD']

for btype in btypes:
#     print("Loading file")
    file = "/hildafs/projects/phy230054p/fep/stroopwafel_dir/stroopwafel/tests/output/{}/fiducial/{}.h5".format(btype, btype)

    k1 = btype[:2].lower()

    if k1 == 'bh':
        m1_low = 5
    elif k1 == 'ns':
        m1_low = 3

#     print("Calculating f_trunc")
    # get the fraction of mass in the cosmic regime
    f_trunc = get_cosmic_fraction(m1_low = m1_low, m1_upp = 150, m2_low = 0.0, f_bin = 0.5)

#     print("Finding metallicity weights")
    # get the metallicity weights
    MW_weights = find_metallicity_weights(bins)

#     print("Final calculations...")
    # get the mass per metallicity bin
    MSF_per_Z, total_DCOs_per_Z = mass_per_Z(file, f_trunc, bins)

    M_MW = 10.4e10

    total_DCOs = np.sum(total_DCOs_per_Z / MSF_per_Z * MW_weights) * M_MW

    print(btype, ': ', total_DCOs)



BHBH :  720445.004898858
BHNS :  332750.13814962853
NSNS :  94124.04600082237
BHWD :  110531.1288602833
NSWD :  4326899.088870831
